## Background
We categorized events by gen level infomartion: boosted, resolved ,semi-boosted, and out of categories (ooc). 

We found most ooc events are in the resolved pT mHH. 

Below is a recap

In [ ]:
from __future__ import annotations

%pip install --upgrade awkward

In [ ]:
import awkward as ak
import hist
import numpy as np
import pandas as pd
import vector

vector.register_awkward()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker

# mplhep for CMS-style plots
import mplhep as hep
from matplotlib.lines import Line2D

plt.style.use(hep.style.CMS)
hep.style.use("CMS")
formatter = mticker.ScalarFormatter(useMathText=True)
formatter.set_powerlimits((-3, 3))
plt.rcParams.update({"font.size": 16})

In [ ]:
def make_vector(events: pd.DataFrame, obj: str):
    """Create a ``vector`` object from the columns of the dataframe"""
    mstring = "Msd" if obj == "ak8FatJet" else "Mass"

    return vector.array(
        {
            "pt": events[f"{obj}Pt"],
            "phi": events[f"{obj}Phi"],
            "eta": events[f"{obj}Eta"],
            "M": events[f"{obj}{mstring}"],
        }
    )

In [ ]:
events = pd.read_parquet(
    "/eos/uscms/store/user/cmantill/bbbb/matching/Oct30/2018/GluGlutoHHto4B_cHHH1_TuneCP5_PSWeights_13TeV-powheg-pythia8/parquet/"
)

In [ ]:
jets = make_vector(events, "ak4Jet")
gen_higgs = make_vector(events, "GenHiggs")
gen_bs = make_vector(events, "Genb")
fjs = make_vector(events, "ak8FatJet")

In [ ]:
list(events.keys())

In [ ]:
drbh1ak8 = events["ak8FatJetMaxdRH1"].to_numpy()
drbh2ak8 = events["ak8FatJetMaxdRH2"].to_numpy()
indexak8 = events["ak8FatJetHiggsMatchIndex"].to_numpy()
indexak4 = events["ak4JetHiggsMatchIndex"].to_numpy()
nbh1ak8 = events["ak8FatJetNumBMatchedH1"].to_numpy()
nbh2ak8 = events["ak8FatJetNumBMatchedH2"].to_numpy()

In [ ]:
# ak4 jets matched to h1 and h2
h1ak4 = indexak4 == 0
h2ak4 = indexak4 == 1
num_ak4m2h1 = h1ak4.sum(axis=1)
num_ak4m2h2 = h2ak4.sum(axis=1)
h1m2ak4 = num_ak4m2h1 == 2
h2m2ak4 = num_ak4m2h2 == 2

# ak8 jets matched to h1 and h2
h1ak8 = indexak8 == 0
h2ak8 = indexak8 == 1
num_ak8m2h1 = h1ak8.sum(axis=1)
num_ak8m2h2 = h2ak8.sum(axis=1)
h1m1ak8 = num_ak8m2h1 == 1
h2m1ak8 = num_ak8m2h2 == 1

boosted = h1m1ak8 & h2m1ak8
resolved = (h1m2ak4 & h2m2ak4) & ~(boosted) & (num_ak8m2h1 == 0) & (num_ak8m2h2 == 0)
semi_resolved_h1 = h1m2ak4 & (h2m1ak8 & ~h2m2ak4) & ~(resolved) & ~(boosted)
semi_resolved_h2 = h2m2ak4 & (h1m1ak8 & ~h1m2ak4) & ~(resolved) & ~(boosted)
semi_resolved = semi_resolved_h1 | semi_resolved_h2
not_categorized = ~(resolved | boosted | semi_resolved)

In [ ]:
h1ak8nb2 = (indexak8 == 0) & (nbh1ak8 == 2)
h2ak8nb2 = (indexak8 == 1) & (nbh2ak8 == 2)
h1m1ak8b2 = h1ak8nb2.sum(axis=1) == 1
h2m1ak8b2 = h2ak8nb2.sum(axis=1) == 1

h1ak8nb1 = (indexak8 == 0) & (nbh1ak8 == 1)
h2ak8nb1 = (indexak8 == 1) & (nbh2ak8 == 1)
h1m1ak8b1 = h1ak8nb1.sum(axis=1) == 1
h2m1ak8b1 = h2ak8nb1.sum(axis=1) == 1

boosted_nb2 = h1m1ak8b2 & h2m1ak8b2
boosted_nb1 = h1m1ak8b1 & h2m1ak8b1
boosted_nb1nb2 = (h1m1ak8b1 & h2m1ak8b2) | (h1m1ak8b2 & h2m1ak8b1)

In [ ]:
semi_resolved_h1_b2 = h1m2ak4 & (h2m1ak8b2 & ~h2m2ak4) & ~(resolved) & ~(boosted)
semi_resolved_h2_b2 = h2m2ak4 & (h1m1ak8b2 & ~h1m2ak4) & ~(resolved) & ~(boosted)
semi_resolved_h1_b1 = h1m2ak4 & (h2m1ak8b1 & ~h2m2ak4) & ~(resolved) & ~(boosted)
semi_resolved_h2_b1 = h2m2ak4 & (h1m1ak8b1 & ~h1m2ak4) & ~(resolved) & ~(boosted)

semi_resolved_h_b2 = semi_resolved_h1_b2 | semi_resolved_h2_b2
semi_resolved_h_b1 = semi_resolved_h1_b1 | semi_resolved_h2_b1

In [ ]:
# calcualte m_HH for every event
mhh = (gen_higgs[:, 0] + gen_higgs[:, 1]).m

# bypass into each category
mhh_boosted = mhh[boosted]
mhh_resolved = mhh[resolved]
mhh_semi = mhh[semi_resolved]
mhh_out = mhh[not_categorized]

# start to plot, code modified on Christina's plot_h1h2_fj
bins = np.arange(mhh.min(), mhh.max(), 10)
bins = np.arange(100, 2000, 50)
var_axis = hist.axis.Variable(bins, name="var", label="variable")
cat_axis = hist.axis.StrCategory([], name="cat", growth=True)

hist_mhh = hist.Hist(var_axis, cat_axis)
hist_mhh.fill(var=mhh_boosted, cat="boosted")
hist_mhh.fill(var=mhh_semi, cat="semi_resolved")
hist_mhh.fill(var=mhh_resolved, cat="resolved")
hist_mhh.fill(var=mhh_out, cat="non-categorized")

leg_elems = []
fig, ax = plt.subplots(1, 1, figsize=(8, 4))
hist_mhh[{"cat": "boosted"}].plot1d(ax=ax, label="Boosted", color="y", ls="-")
hist_mhh[{"cat": "semi_resolved"}].plot1d(ax=ax, label="Semi-Resolved", color="r", ls="-.")
hist_mhh[{"cat": "resolved"}].plot1d(ax=ax, label="Resolved", color="k", ls=":")
hist_mhh[{"cat": "non-categorized"}].plot1d(
    ax=ax, label="Outside of these categories", color="c", ls="--"
)
leg_elems.append(Line2D([0], [0], color="y", lw=2, ls="-", label="boosted"))
leg_elems.append(Line2D([0], [0], color="r", lw=2, ls="-.", label="semi-resolved"))
leg_elems.append(Line2D([0], [0], color="k", lw=2, ls=":", label="resolved"))
leg_elems.append(Line2D([0], [0], color="c", lw=2, ls="--", label="ooc"))
leg = ax.legend(handles=leg_elems)
leg.set_title("Gen-Level categories", prop={"size": 10})
ax.set_xlim(0, 2000)
ax.set_ylim(0, 30000)
ax.set_xlabel(r"$m_{HH}$")
ax.set_title("Catrgories of RecoJet->GenH Matching (AK8=AK4)")

## Investigations
### Ak4 matched to b quarks vs. Higgs pT

In [ ]:
events_ooc = events[not_categorized]
Hs_ooc = gen_higgs[not_categorized]
js_ooc = jets[not_categorized]

In [ ]:
b_m2h1_ooc = gen_bs[not_categorized][:, 0:2]
b_m2h2_ooc = gen_bs[not_categorized][:, 2:4]

In [ ]:
# calculate dR(b_quark, jets)
dR_h1b1_js = b_m2h1_ooc[:, 0][:, np.newaxis].deltaR(js_ooc)
dR_h1b2_js = b_m2h1_ooc[:, 1][:, np.newaxis].deltaR(js_ooc)
dR_h2b1_js = b_m2h2_ooc[:, 0][:, np.newaxis].deltaR(js_ooc)
dR_h2b2_js = b_m2h2_ooc[:, 1][:, np.newaxis].deltaR(js_ooc)

# count how many jets are matched to each b
N_jm2b1h1 = ak.sum(dR_h1b1_js < 0.5, axis=-1)
N_jm2b2h1 = ak.sum(dR_h1b2_js < 0.5, axis=-1)
N_jm2b1h2 = ak.sum(dR_h2b1_js < 0.5, axis=-1)
N_jm2b2h2 = ak.sum(dR_h2b2_js < 0.5, axis=-1)

In [ ]:
# categorize them into
# 1. exceed: both b1 and b2 matched to more than 1 ak4
# 2. partial_exceed: one of the bs is matched to more than 1 ak4, another is matched to 1 ak4
# 3. polarized: one of the bs is matched to more than 1 ak4, another is unmatched
# 4. match: both b1 and b2 are uniquely matched
# 5. partial_match: only one of the b is uniquely matched, the other is unmatched
# 6. unmatch: no b is matched
h1_exceed = (N_jm2b1h1 > 1) & (N_jm2b2h1 > 1)
h2_exceed = (N_jm2b1h2 > 1) & (N_jm2b2h2 > 1)

h1_partial_exceed = ((N_jm2b1h1 > 1) & (N_jm2b2h1 == 1)) | ((N_jm2b1h1 == 1) & (N_jm2b2h1 > 1))
h2_partial_exceed = ((N_jm2b1h2 > 1) & (N_jm2b2h2 == 1)) | ((N_jm2b1h2 == 1) & (N_jm2b2h2 > 1))

h1_polarized = ((N_jm2b1h1 > 1) & (N_jm2b2h1 == 0)) | ((N_jm2b1h1 == 0) & (N_jm2b2h1 > 1))
h2_polarized = ((N_jm2b1h2 > 1) & (N_jm2b2h2 == 0)) | ((N_jm2b1h2 == 0) & (N_jm2b2h2 > 1))

h1_match = (N_jm2b1h1 == 1) & (N_jm2b2h1 == 1)
h2_match = (N_jm2b1h2 == 1) & (N_jm2b2h2 == 1)

h1_partial_match = ((N_jm2b1h1 == 1) & (N_jm2b2h1 == 0)) | ((N_jm2b1h1 == 0) & (N_jm2b2h1 == 1))
h2_partial_match = ((N_jm2b1h2 == 1) & (N_jm2b2h2 == 0)) | ((N_jm2b1h2 == 0) & (N_jm2b2h2 == 1))

h1_unmatch = (N_jm2b1h1 == 0) & (N_jm2b2h1 == 0)
h2_unmatch = (N_jm2b1h2 == 0) & (N_jm2b2h2 == 0)

In [ ]:
# check if the event numbers in each category adds up to total number of events
category_h1 = [h1_exceed, h1_partial_exceed, h1_match, h1_polarized, h1_partial_match, h1_unmatch]
category_h2 = [h2_exceed, h2_partial_exceed, h2_match, h2_polarized, h2_partial_match, h2_unmatch]
category = category_h1 + category_h2

N_tot_running = 0
for cat in category_h1:
    N_tot_running += ak.sum(cat)

print(N_tot_running, events_ooc.shape[0])

In [ ]:
ooc_ooc = (
    (~h1_exceed)
    & (~h1_partial_exceed)
    & (~h1_match)
    & (~h1_polarized)
    & (~h1_partial_match)
    & (~h1_unmatch)
)

In [ ]:
for Ns in zip(N_jm2b1h1[ooc_ooc], N_jm2b2h1[ooc_ooc], N_jm2b1h2[ooc_ooc], N_jm2b2h2[ooc_ooc]):
    print(Ns)

ak.sum(ooc_ooc)

In [ ]:
# get pt in each H each category
H1 = Hs_ooc[:, 0]
h1pt_exceed = H1[h1_exceed].pt
h1pt_partial_exceed = H1[h1_partial_exceed].pt
h1pt_polzrized = H1[h1_polarized].pt
h1pt_match = H1[h1_match].pt
h1pt_partial_match = H1[h1_partial_match].pt
h1pt_unmatch = H1[h1_unmatch].pt

In [ ]:
H2 = Hs_ooc[:, 1]
h2pt_exceed = H2[h2_exceed].pt
h2pt_partial_exceed = H2[h2_partial_exceed].pt
h2pt_polzrized = H2[h2_polarized].pt
h2pt_match = H2[h2_match].pt
h2pt_partial_match = H2[h2_partial_match].pt
h2pt_unmatch = H2[h2_unmatch].pt

In [ ]:
# start to plot
bins = np.arange(0, 500, 10)
var_axis = hist.axis.Variable(bins, name="var", label="variable")
cat_axis = hist.axis.StrCategory([], name="cat", growth=True)

hist_h1pt = hist.Hist(var_axis, cat_axis)
hist_h1pt.fill(var=h1pt_exceed, cat="exceed")
hist_h1pt.fill(var=h1pt_partial_exceed, cat="partial_exceed")
hist_h1pt.fill(var=h1pt_polzrized, cat="polarized")
hist_h1pt.fill(var=h1pt_match, cat="match")
hist_h1pt.fill(var=h1pt_partial_match, cat="partial_match")
hist_h1pt.fill(var=h1pt_unmatch, cat="unmatch")

hist_h2pt = hist.Hist(var_axis, cat_axis)
hist_h2pt.fill(var=h2pt_exceed, cat="exceed")
hist_h2pt.fill(var=h2pt_partial_exceed, cat="partial_exceed")
hist_h2pt.fill(var=h2pt_polzrized, cat="polarized")
hist_h2pt.fill(var=h2pt_match, cat="match")
hist_h2pt.fill(var=h2pt_partial_match, cat="partial_match")
hist_h2pt.fill(var=h2pt_unmatch, cat="unmatch")


fig, ax = plt.subplots(1, 2, figsize=(16, 4))
hist_h1pt[{"cat": "exceed"}].plot1d(ax=ax[0], label="exceed")
hist_h1pt[{"cat": "partial_exceed"}].plot1d(ax=ax[0], label="partial_exceed")
hist_h1pt[{"cat": "polarized"}].plot1d(ax=ax[0], label="polarized")
hist_h1pt[{"cat": "match"}].plot1d(ax=ax[0], label="match")
hist_h1pt[{"cat": "partial_match"}].plot1d(ax=ax[0], label="partial_match")
hist_h1pt[{"cat": "unmatch"}].plot1d(ax=ax[0], label="unmatch")

hist_h2pt[{"cat": "exceed"}].plot1d(ax=ax[1], label="exceed")
hist_h2pt[{"cat": "partial_exceed"}].plot1d(ax=ax[1], label="partial_exceed")
hist_h2pt[{"cat": "polarized"}].plot1d(ax=ax[1], label="polarized")
hist_h2pt[{"cat": "match"}].plot1d(ax=ax[1], label="match")
hist_h2pt[{"cat": "partial_match"}].plot1d(ax=ax[1], label="partial_match")
hist_h2pt[{"cat": "unmatch"}].plot1d(ax=ax[1], label="unmatch")

leg1 = ax[0].legend(fontsize=10)
leg1.set_title("b-AK4 match categories", prop={"size": 10})
ax[0].set_xlim(0, 500)
ax[0].set_xlabel(r"$pT_{H_1}$")

leg2 = ax[1].legend(fontsize=10)
leg2.set_title("b-AK4 match categories", prop={"size": 10})
ax[1].set_xlim(0, 500)
ax[1].set_xlabel(r"$pT_{H_2}$")

### OOC matching combination statistics

In [ ]:
def parse_2d_num_pairs(array, sym=False):
    if sym is True:
        for pair in array:
            print("")
            print(f"One H matched to {pair[0][0]} AK4 jets and {pair[1][0]} AK8 jets")
            print(f"The other H matched to {pair[0][1]} AK4 jets and {pair[1][1]} AK8 jets")
    else:
        for pair in array:
            print("")
            print(f"H1 matched to {pair[0][0]} AK4 jets and {pair[1][0]} AK8 jets")
            print(f"H2 matched to {pair[0][1]} AK4 jets and {pair[1][1]} AK8 jets")

    return

In [ ]:
# e.g. number of ak4 jets matched to h1 and h2 for non-categorized events
num_ak4N8m2h_nc = (
    np.stack(
        [
            num_ak4m2h1[not_categorized],
            num_ak4m2h2[not_categorized],
            num_ak8m2h1[not_categorized],
            num_ak8m2h2[not_categorized],
        ],
        axis=1,
    )
    .reshape(-1, 2, 2)
    .tolist()
)

# store all recorded combinations of [[num_ak4m2H1, num_ak4m2H2], [num_ak8m2H1, num_ak8m2H2]]
pair_ak4N8m2h_nc = []

# loop from all num pairs constructed from the ooc events
for pair in num_ak4N8m2h_nc:
    if pair not in pair_ak4N8m2h_nc:
        pair_ak4N8m2h_nc.append(pair)

parse_2d_num_pairs(pair_ak4N8m2h_nc)

In [ ]:
print(
    num_ak4m2h1[not_categorized][0:3],
    num_ak4m2h2[not_categorized][0:3],
    num_ak8m2h1[not_categorized][0:3],
    num_ak8m2h2[not_categorized][0:3],
)

In [ ]:
np.stack(
    [
        num_ak4m2h1[not_categorized][0:3],
        num_ak4m2h2[not_categorized][0:3],
        num_ak8m2h1[not_categorized][0:3],
        num_ak8m2h2[not_categorized][0:3],
    ],
    axis=1,
)

In [ ]:
np.stack(
    [
        num_ak4m2h1[not_categorized][0:3],
        num_ak4m2h2[not_categorized][0:3],
        num_ak8m2h1[not_categorized][0:3],
        num_ak8m2h2[not_categorized][0:3],
    ],
    axis=1,
).reshape(-1, 2, 2).tolist()

In [ ]:
# This cell is tryig to assume H1 and H2 is symmetric
# e.g. num_ak4m2HX = (2, 1) == (1, 2)
# the point is to not differentiate H1 and H2
# note that if ak4 pair is permutated, then ak8 pair should be permutated in the same way

# store all permutations of the recorded combinations of [[num_ak4m2H1, num_ak4m2H2], [num_ak8m2H1, num_ak8m2H2]]
perm_pair_ak4N8m2h_nc = []
# store unique combinations of [num_ak4m2HX, num_ak8m2HX]
unique_pair_ak4N8m2h_nc = []

# loop from all num pairs constructed from the ooc events
for pair in num_ak4N8m2h_nc:
    # if it doesn't match any permutation of the recorded unique pair
    if pair not in perm_pair_ak4N8m2h_nc:
        # add to the unique pair
        unique_pair_ak4N8m2h_nc.append(pair)
        # also add its permutations to the permutation list
        perm_pair_ak4N8m2h_nc.append(pair)
        perm_pair_ak4N8m2h_nc.append([pair[0][::-1], pair[1][::-1]])

parse_2d_num_pairs(unique_pair_ak4N8m2h_nc, sym=True)

In [ ]:
print(unique_pair_ak4N8m2h_nc)

In [ ]:
# construct a dictionary to store ooc events in each H matching pattern:
LUT = {}
for pair in unique_pair_ak4N8m2h_nc:
    num_ak4m2h1 = pair[0][0]
    num_ak4m2h2 = pair[0][1]
    num_ak8m2h1 = pair[1][0]
    num_ak8m2h2 = pair[1][1]
    LUT[f"{num_ak4m2h1}{num_ak4m2h2}{num_ak8m2h1}{num_ak8m2h2}"] = 0

In [ ]:
for num in num_ak4N8m2h_nc:
    num_ak4m2h1 = num[0][0]
    num_ak4m2h2 = num[0][1]
    num_ak8m2h1 = num[1][0]
    num_ak8m2h2 = num[1][1]

    if num in unique_pair_ak4N8m2h_nc:
        LUT[f"{num_ak4m2h1}{num_ak4m2h2}{num_ak8m2h1}{num_ak8m2h2}"] += 1
    else:
        LUT[f"{num_ak4m2h2}{num_ak4m2h1}{num_ak8m2h2}{num_ak8m2h1}"] += 1

In [ ]:
labels_pie = np.array(list(LUT.keys()))
ys_pie = np.array(list(LUT.values()))
labels_pie = labels_pie[np.argsort(ys_pie)[::-1]]
ys_pie = ys_pie[np.argsort(ys_pie)[::-1]]

In [ ]:
print(ys_pie)

In [ ]:
fig, ax = plt.subplots()
ax.pie(ys_pie, labels=labels_pie, autopct="%1.0f%%")
ax.set(title="Pie Chart of OOC AK4RecoJet Matching Patterns (AK8=AK4)")

fig.text(0.5, 0.85, "Number convention: AK4H1, AK4H2, AK8H1 AK8H2", ha="center", c="b")
plt.show()

# Number convention: AK4H1, AK4H2, AK8H1 AK8H2

## Now change matching criteria to recoJet->genJet->genb

In [ ]:
genJs = make_vector(events, "ak4GenJet")
genBs = make_vector(events, "Genb")

In [ ]:
# dissemble gen_b by H
b_m2h1 = genBs[:, 0:2]
b_m2h2 = genBs[:, 2:4]

In [ ]:
# calculate dR between genb and each genJet
dR_h1b1_genJs = b_m2h1[:, 0][:, np.newaxis].deltaR(genJs)
dR_h1b2_genJs = b_m2h1[:, 1][:, np.newaxis].deltaR(genJs)
dR_h2b1_genJs = b_m2h2[:, 0][:, np.newaxis].deltaR(genJs)
dR_h2b2_genJs = b_m2h2[:, 1][:, np.newaxis].deltaR(genJs)

In [ ]:
# count how many jets are matched to each b
GenJm2b1h1 = (dR_h1b1_genJs < 0.4)[:, :, np.newaxis]
GenJm2b2h1 = (dR_h1b2_genJs < 0.4)[:, :, np.newaxis]
GenJm2b1h2 = (dR_h2b1_genJs < 0.4)[:, :, np.newaxis]
GenJm2b2h2 = (dR_h2b2_genJs < 0.4)[:, :, np.newaxis]
print(GenJm2b1h1.shape)

# concatenate the arrays
# 1st dimension: event
# 2nd dimension: GenJ
# 3rd dimension: match to each b true or false
GenJm2b = np.concatenate([GenJm2b1h1, GenJm2b2h1, GenJm2b1h2, GenJm2b2h2], axis=2)
print(GenJm2b.shape)

In [ ]:
# construct ak4GenJet info
ak4GenJetHiggsMatch = np.zeros(shape=genJs.shape, dtype=bool)
ak4GenJetHiggsMatchIndex = -1 * np.ones(shape=genJs.shape, dtype=int)

# If a gen J is matched to 1 and only 1 genb,
# it is considered to be matched to the mother H
# and will be assigned with a ak4GenJetHiggsMatchIndex
GenJm2H = np.sum(GenJm2b, axis=2) == 1
ak4GenJetHiggsMatch[GenJm2H] = True

# find which genb and H matched the genJ
# draft in the sense that only the matched genJ's
# genB and genH indices will be used
bIdx_draft = np.argmax(GenJm2b, axis=2)
hIdx_draft = np.floor(bIdx_draft / 2)
ak4GenJetHiggsMatchIndex[ak4GenJetHiggsMatch] = hIdx_draft[ak4GenJetHiggsMatch]

### Plotting each categories for the genJ genb matching

In [ ]:
indexak8 = events["ak8FatJetHiggsMatchIndex"].to_numpy()
indexak4 = ak4GenJetHiggsMatchIndex

In [ ]:
# ak4 jets matched to h1 and h2
h1ak4 = indexak4 == 0
h2ak4 = indexak4 == 1
num_ak4m2h1 = h1ak4.sum(axis=1)
num_ak4m2h2 = h2ak4.sum(axis=1)
h1m2ak4 = num_ak4m2h1 == 2
h2m2ak4 = num_ak4m2h2 == 2

# ak8 jets matched to h1 and h2
h1ak8 = indexak8 == 0
h2ak8 = indexak8 == 1
num_ak8m2h1 = h1ak8.sum(axis=1)
num_ak8m2h2 = h2ak8.sum(axis=1)
h1m1ak8 = num_ak8m2h1 == 1
h2m1ak8 = num_ak8m2h2 == 1

boosted = h1m1ak8 & h2m1ak8
resolved = (h1m2ak4 & h2m2ak4) & ~(boosted) & (num_ak8m2h1 == 0) & (num_ak8m2h2 == 0)
semi_resolved_h1 = h1m2ak4 & h2m1ak8 & ~(resolved) & ~(boosted)
semi_resolved_h2 = h2m2ak4 & h1m1ak8 & ~(resolved) & ~(boosted)
semi_resolved = semi_resolved_h1 | semi_resolved_h2
not_categorized = ~(resolved | boosted | semi_resolved)

In [ ]:
h1ak8nb2 = (indexak8 == 0) & (nbh1ak8 == 2)
h2ak8nb2 = (indexak8 == 1) & (nbh2ak8 == 2)
h1m1ak8b2 = h1ak8nb2.sum(axis=1) == 1
h2m1ak8b2 = h2ak8nb2.sum(axis=1) == 1

h1ak8nb1 = (indexak8 == 0) & (nbh1ak8 == 1)
h2ak8nb1 = (indexak8 == 1) & (nbh2ak8 == 1)
h1m1ak8b1 = h1ak8nb1.sum(axis=1) == 1
h2m1ak8b1 = h2ak8nb1.sum(axis=1) == 1

boosted_nb2 = h1m1ak8b2 & h2m1ak8b2
boosted_nb1 = h1m1ak8b1 & h2m1ak8b1
boosted_nb1nb2 = (h1m1ak8b1 & h2m1ak8b2) | (h1m1ak8b2 & h2m1ak8b1)

In [ ]:
semi_resolved_h1_b2 = h1m2ak4 & (h2m1ak8b2 & ~h2m2ak4) & ~(resolved) & ~(boosted)
semi_resolved_h2_b2 = h2m2ak4 & (h1m1ak8b2 & ~h1m2ak4) & ~(resolved) & ~(boosted)
semi_resolved_h1_b1 = h1m2ak4 & (h2m1ak8b1 & ~h2m2ak4) & ~(resolved) & ~(boosted)
semi_resolved_h2_b1 = h2m2ak4 & (h1m1ak8b1 & ~h1m2ak4) & ~(resolved) & ~(boosted)

semi_resolved_h_b2 = semi_resolved_h1_b2 | semi_resolved_h2_b2
semi_resolved_h_b1 = semi_resolved_h1_b1 | semi_resolved_h2_b1

In [ ]:
# calcualte m_HH for every event
mhh = (gen_higgs[:, 0] + gen_higgs[:, 1]).m

# bypass into each category
mhh_boosted = mhh[boosted]
mhh_resolved = mhh[resolved]
mhh_semi = mhh[semi_resolved]
mhh_out = mhh[not_categorized]

# start to plot, code modified on Christina's plot_h1h2_fj
bins = np.arange(mhh.min(), mhh.max(), 10)
bins = np.arange(100, 2000, 50)
var_axis = hist.axis.Variable(bins, name="var", label="variable")
cat_axis = hist.axis.StrCategory([], name="cat", growth=True)

hist_mhh = hist.Hist(var_axis, cat_axis)
hist_mhh.fill(var=mhh_boosted, cat="boosted")
hist_mhh.fill(var=mhh_semi, cat="semi_resolved")
hist_mhh.fill(var=mhh_resolved, cat="resolved")
hist_mhh.fill(var=mhh_out, cat="non-categorized")

leg_elems = []
fig, ax = plt.subplots(1, 1, figsize=(8, 4))
hist_mhh[{"cat": "boosted"}].plot1d(ax=ax, label="Boosted", color="y", ls="-")
hist_mhh[{"cat": "semi_resolved"}].plot1d(ax=ax, label="Semi-Resolved", color="r", ls="-.")
hist_mhh[{"cat": "resolved"}].plot1d(ax=ax, label="Resolved", color="k", ls=":")
hist_mhh[{"cat": "non-categorized"}].plot1d(
    ax=ax, label="Outside of these categories", color="c", ls="--"
)
leg_elems.append(Line2D([0], [0], color="y", lw=2, ls="-", label="boosted"))
leg_elems.append(Line2D([0], [0], color="r", lw=2, ls="-.", label="semi-resolved"))
leg_elems.append(Line2D([0], [0], color="k", lw=2, ls=":", label="resolved"))
leg_elems.append(Line2D([0], [0], color="c", lw=2, ls="--", label="ooc"))
leg = ax.legend(handles=leg_elems)
leg.set_title("Gen-Level categories", prop={"size": 10})
ax.set_xlim(0, 2000)
ax.set_ylim(0, 30000)
ax.set_xlabel(r"$m_{HH}$")
ax.set_title("Catrgories of GenJet->GenH Matching (AK8=AK4)")

### Mapping genJ to recoJ after having more information in the dataset

In [ ]:
# get ak4JetGenJetIdx
# to know genJet recoJet correspondence
ak4JetGenJetIdx = events["ak4JetGenJetIdx"].to_numpy()
not_padded = ak4JetGenJetIdx > 0
not_oor = ak4JetGenJetIdx < 6  # oor=out of range
select = not_padded & not_oor

In [ ]:
ak4JetGenJetIdx_select = ak.mask(ak4JetGenJetIdx, select)
ak4JetGenJetIdx_select = ak.drop_none(ak4JetGenJetIdx_select)

In [ ]:
# select the genJet Higgs matching index that genJet also matched to reco jet
ak4GenJetHiggsMatchIndex_ak = ak.Array(ak4GenJetHiggsMatchIndex)
ak4JetHiggsMatchIndex_new = ak4GenJetHiggsMatchIndex_ak[ak4JetGenJetIdx_select]
# because we only care how many reco jet matched to h1 and h2
# the order of reco jet in each event is irrelevant

# pad each event to 6
ak4JetHiggsMatchIndex_new = ak.fill_none(
    ak.pad_none(ak4JetHiggsMatchIndex_new, 6, axis=-1), -1
).to_numpy()

In [ ]:
indexak8 = events["ak8FatJetHiggsMatchIndex"].to_numpy()
indexak4 = ak4JetHiggsMatchIndex_new

In [ ]:
# ak4 jets matched to h1 and h2
h1ak4 = indexak4 == 0
h2ak4 = indexak4 == 1
num_ak4m2h1 = h1ak4.sum(axis=1)
num_ak4m2h2 = h2ak4.sum(axis=1)
h1m2ak4 = num_ak4m2h1 == 2
h2m2ak4 = num_ak4m2h2 == 2

# ak8 jets matched to h1 and h2
h1ak8 = indexak8 == 0
h2ak8 = indexak8 == 1
num_ak8m2h1 = h1ak8.sum(axis=1)
num_ak8m2h2 = h2ak8.sum(axis=1)
h1m1ak8 = num_ak8m2h1 == 1
h2m1ak8 = num_ak8m2h2 == 1

boosted = h1m1ak8 & h2m1ak8
resolved = (h1m2ak4 & h2m2ak4) & ~(boosted) & (num_ak8m2h1 == 0) & (num_ak8m2h2 == 0)
semi_resolved_h1 = h1m2ak4 & (h2m1ak8 & ~h2m2ak4) & ~(resolved) & ~(boosted)
semi_resolved_h2 = h2m2ak4 & (h1m1ak8 & ~h1m2ak4) & ~(resolved) & ~(boosted)
semi_resolved = semi_resolved_h1 | semi_resolved_h2
not_categorized = ~(resolved | boosted | semi_resolved)

In [ ]:
h1ak8nb2 = (indexak8 == 0) & (nbh1ak8 == 2)
h2ak8nb2 = (indexak8 == 1) & (nbh2ak8 == 2)
h1m1ak8b2 = h1ak8nb2.sum(axis=1) == 1
h2m1ak8b2 = h2ak8nb2.sum(axis=1) == 1

h1ak8nb1 = (indexak8 == 0) & (nbh1ak8 == 1)
h2ak8nb1 = (indexak8 == 1) & (nbh2ak8 == 1)
h1m1ak8b1 = h1ak8nb1.sum(axis=1) == 1
h2m1ak8b1 = h2ak8nb1.sum(axis=1) == 1

boosted_nb2 = h1m1ak8b2 & h2m1ak8b2
boosted_nb1 = h1m1ak8b1 & h2m1ak8b1
boosted_nb1nb2 = (h1m1ak8b1 & h2m1ak8b2) | (h1m1ak8b2 & h2m1ak8b1)

In [ ]:
semi_resolved_h1_b2 = h1m2ak4 & (h2m1ak8b2 & ~h2m2ak4) & ~(resolved) & ~(boosted)
semi_resolved_h2_b2 = h2m2ak4 & (h1m1ak8b2 & ~h1m2ak4) & ~(resolved) & ~(boosted)
semi_resolved_h1_b1 = h1m2ak4 & (h2m1ak8b1 & ~h2m2ak4) & ~(resolved) & ~(boosted)
semi_resolved_h2_b1 = h2m2ak4 & (h1m1ak8b1 & ~h1m2ak4) & ~(resolved) & ~(boosted)

semi_resolved_h_b2 = semi_resolved_h1_b2 | semi_resolved_h2_b2
semi_resolved_h_b1 = semi_resolved_h1_b1 | semi_resolved_h2_b1

In [ ]:
# calcualte m_HH for every event
mhh = (gen_higgs[:, 0] + gen_higgs[:, 1]).m

# bypass into each category
mhh_boosted = mhh[boosted]
mhh_resolved = mhh[resolved]
mhh_semi = mhh[semi_resolved]
mhh_out = mhh[not_categorized]

# start to plot, code modified on Christina's plot_h1h2_fj
bins = np.arange(mhh.min(), mhh.max(), 10)
bins = np.arange(100, 2000, 50)
var_axis = hist.axis.Variable(bins, name="var", label="variable")
cat_axis = hist.axis.StrCategory([], name="cat", growth=True)

hist_mhh = hist.Hist(var_axis, cat_axis)
hist_mhh.fill(var=mhh_boosted, cat="boosted")
hist_mhh.fill(var=mhh_semi, cat="semi_resolved")
hist_mhh.fill(var=mhh_resolved, cat="resolved")
hist_mhh.fill(var=mhh_out, cat="non-categorized")

leg_elems = []
fig, ax = plt.subplots(1, 1, figsize=(8, 4))
hist_mhh[{"cat": "boosted"}].plot1d(ax=ax, label="Boosted", color="y", ls="-")
hist_mhh[{"cat": "semi_resolved"}].plot1d(ax=ax, label="Semi-Resolved", color="r", ls="-.")
hist_mhh[{"cat": "resolved"}].plot1d(ax=ax, label="Resolved", color="k", ls=":")
hist_mhh[{"cat": "non-categorized"}].plot1d(
    ax=ax, label="Outside of these categories", color="c", ls="--"
)
leg_elems.append(Line2D([0], [0], color="y", lw=2, ls="-", label="boosted"))
leg_elems.append(Line2D([0], [0], color="r", lw=2, ls="-.", label="semi-resolved"))
leg_elems.append(Line2D([0], [0], color="k", lw=2, ls=":", label="resolved"))
leg_elems.append(Line2D([0], [0], color="c", lw=2, ls="--", label="ooc"))
leg = ax.legend(handles=leg_elems)
leg.set_title("Gen-Level categories", prop={"size": 10})
ax.set_xlim(0, 2000)
ax.set_ylim(0, 30000)
ax.set_xlabel(r"$m_{HH}$")
ax.set_title("Catrgories of RecoJet->GenJet->GenH Matching (AK8=AK4)")